# Tracking Sensor Bias



In [ ]:
from collections import OrderedDict

import pyro
import pyro.distributions as dist

import funsor
import funsor.distributions as f_dist
import funsor.ops as ops
from funsor.interpreter import interpretation, reinterpret
from funsor.optimizer import apply_optimizer
from funsor.terms import lazy

Simulate some synthetic data:

In [ ]:
num_sensors = 5
num_steps = 100

# simulate biased sensors
sensors  = []
for _ in range(num_sensors):
    true_pos = 10 * torch.rand(2)  # in a box
    biased_pos = true_pos + torch.randn(2)
    sensors.append({
        "true_pos": true_pos,
        "biased_pos": biased_pos,
    })

# simulate a single track
track = []
z = 10 * torch.rand(2)  # initial state
for t in range(num_steps):
    # Advance latent state.
    z += torch.randn(2)
    z.clamp_(min=0, max=10)  # keep in the box
    
    # Observe via a random sensor.
    sensor_id = pyro.sample('id', dist.Categorical(torch.ones(num_sensors)))
    x = z - sensors[sensor_id]["true_pos"] + torch.randn(2)
    track.append({"sensor_id": sensor_id, "x": x})

Now let's set up a tracking problem in Funsor. We start by modeling the biases of each sensor.

In [ ]:
# FIXME
bias = funsor.pyro.mvn_to_funsor(
    dist.MultivariateNormal(
        torch.zeros(num_sensors, 2),
        torch.eye(2, requires_grad=True)  # This can be learned
    ),
    ("sensor_id",),  # a batch dim
)(value="bias")
# Instead create a giant joint Gaussian:
bias = sum(
    funsor.pyro.mvn_to_funsor(
        dist.MultivariateNormal(
            torch.zeros(2),
            torch.eye(2, requires_grad=True)  # This can be learned
        )
    )(value="bias_{}".format(i))
    for i in range(num_sensors)
)

Set up the filter in funsor.

In [ ]:
# Similar to funsor.pyro.hmm.GaussianHMM.__init__()
init_dist = dist.MultivariateNormal(TODO)
trans_matrix = TODO
trans_dist = dist.MultivariateNormal(
    torch.zeros(2),
    torch.eye(2))
obs_matrix = torch.eye(2)
obs_dist = dist.MultivariateNormal(
    torch.zeros(2),
    torch.eye(2))

init = dist_to_funsor(initial_dist)(value="state")
trans = matrix_and_mvn_to_funsor(transition_matrix, transition_dist,
                                 ("time",), "state", "state(time=1)")
obs = matrix_and_mvn_to_funsor(observation_matrix, observation_dist,
                               ("time",), "state(time=1)", "value")
# Now this is the crux, we add bias to the observation
sensor_ids = funsor.torch.Tensor(
    torch.tensor([frame["sensor_id"] for frame in track]),
    OrderedDict([("sensor_id": bint(num_frames))]),
    dtype=len(sensors)
)
biased_observations = funsor.torch.Tensor(
    torch.tensor([frame["obs"] for frame in track]),
    OrderedDict([("sensor_id": bint(num_frames))])
)
bias_over_time = bias(sensor_id=sensor_ids)
obs = obs(value=TODO)

In [ ]:
# Similar to funsor.pyro.hmm.GaussianHMM.log_prob()
ndims = max(len(self.batch_shape), value.dim() - self.event_dim)
value = tensor_to_funsor(value, ("time",), event_output=self.event_dim - 1,
                         dtype=self.dtype)

obs = self._obs(value=value)
result = self._trans + obs
result = sequential_sum_product(ops.logaddexp, ops.add,
                                result, "time", {"state": "state(time=1)"})
result += self._init
result = result.reduce(ops.logaddexp, frozenset(["state", "state(time=1)"]))

## Inference

Finally we have a result that is a joint Gaussian over the biases.
We can
1. optimize all parameters to maximize `result.reduce(obs.logaddexp)`
2. estimate the joint distribution over all bias parameters.